# Constants

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4"
project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    FileSearchTool,
)
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

# Check if we have any files already on Azure AI Foundry
If needed, we may delete them using the cells at the bottom of this notebook

In [3]:
project_client.agents.list_files()['data']

[]

# Upload file(s) to search into

In [4]:
file = project_client.agents.upload_file_and_poll(file_path="./product_info_1.md", purpose="assistants")
print(f"Uploaded file: {file}")

Uploaded file: {'object': 'file', 'id': 'assistant-zADxredP0oxMQQGsHwR6cOcq', 'purpose': 'assistants', 'filename': 'product_info_1.md', 'bytes': 10999, 'created_at': 1737281181, 'status': 'processed', 'status_details': None}


# Create a vector store to vectorize file(s)

In [5]:
vector_store = project_client.agents.create_vector_store_and_poll(file_ids=[file.id], name="my_vectorstore")
print(f'Created vector store <{vector_store}>\n\n...and uploaded file "{file.filename}".')

Created vector store <{'id': 'vs_X2aflqg4DzzvhHhEjpgID5Q9', 'object': 'vector_store', 'name': 'my_vectorstore', 'status': 'completed', 'usage_bytes': 17143, 'created_at': 1737281183, 'file_counts': {'in_progress': 0, 'completed': 1, 'failed': 0, 'cancelled': 0, 'total': 1}, 'metadata': {}, 'expires_after': None, 'expires_at': None, 'last_active_at': 1737281183}>

...and uploaded file "product_info_1.md".


# Create file search tool with resources followed by creating agent

In [6]:
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

print(f"file_search.definitions: {file_search_tool.definitions}")
print(f"file_search.resources: {file_search_tool.resources}")

file_search.definitions: [{'type': 'file_search'}]
file_search.resources: {'file_search': {'vector_store_ids': ['vs_X2aflqg4DzzvhHhEjpgID5Q9']}}


# Create AI Foundry Agent

In [7]:
agent = project_client.agents.create_agent(
    model=model_name,
    name="file-agent",
    instructions="Hello, you are helpful agent that can search information from uploaded files",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_M5FwpTjyuRnmnFbV9Wq0jxLG


# Create the thread and attach a new message to it

In [8]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="Hello, how much for the TrailMaster?",
)
print(f"Created message: {message}")

Created thread: {'id': 'thread_MI1ycGHSwqCvrxSomNDuGxBA', 'object': 'thread', 'created_at': 1737281186, 'metadata': {}, 'tool_resources': {}}

Created message: {'id': 'msg_fAYrT3IxlJvpRnBuEgvM2Udz', 'object': 'thread.message', 'created_at': 1737281186, 'assistant_id': None, 'thread_id': 'thread_MI1ycGHSwqCvrxSomNDuGxBA', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, how much for the TrailMaster?', 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent syncrhonously

In [9]:
%%time
# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, assistant_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED.

Run: {'id': 'run_AyEzTyd3TatqWHZ5Q0KLd4k2', 'object': 'thread.run', 'created_at': 1737281187, 'assistant_id': 'asst_M5FwpTjyuRnmnFbV9Wq0jxLG', 'thread_id': 'thread_MI1ycGHSwqCvrxSomNDuGxBA', 'status': 'completed', 'started_at': 1737281187, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1737281193, 'required_action': None, 'last_error': None, 'model': 'gpt-4', 'instructions': 'Hello, you are helpful agent that can search information from uploaded files', 'tools': [{'type': 'file_search', 'file_search': {'ranking_options': {'ranker': 'default_2024_08_21', 'score_threshold': 0.0}}}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 5842, 'completion_tokens': 36, 'total_tokens': 5878}, 'response_format': 'auto', 'too

# Fetch messages from the thread after the agent run execution

In [10]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages, starting with the most recent one:\n")
    i=0
    for m in messages.data:
        j = 0
        i += 1
        print(f"\n===== MESSAGE {messages_nr-i+1} =====")
        for c in m.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages, starting with the most recent one:


===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: The price for the TrailMaster X4 Tent is $250【4:0†source】.
>>> Annotation in MessageTextContent 1 of message 1: 【4:0†source】

>>> Annotation in MessageTextContent 1 of message 1: 【4:0†source】


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: Hello, how much for the TrailMaster?


## Fetch citations from the messages

In [11]:
for citation in messages.file_citation_annotations:
    print(citation)

{'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 45, 'end_index': 57, 'file_citation': {'file_id': 'assistant-zADxredP0oxMQQGsHwR6cOcq'}}
{'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 45, 'end_index': 57, 'file_citation': {'file_id': 'assistant-zADxredP0oxMQQGsHwR6cOcq'}}


# START teardown

In [12]:
# Delete all files

print(f"{len(project_client.agents.list_files()['data'])} files will now be deleted")

i=0
for f in project_client.agents.list_files()['data']:
    i += 1
    project_client.agents.delete_file(file_id=f.id)
    print(f"\n{i} - file {f.filename} has been deleted")

1 files will now be deleted

1 - file product_info_1.md has been deleted


In [13]:
# Delete all vector stores

all_vector_stores = project_client.agents.list_vector_stores()['data']

print(f"{len(all_vector_stores)} vector stores will now be deleted")

i=0
for vs in all_vector_stores:
    i += 1
    project_client.agents.delete_vector_store(vector_store_id=vs.id)
    print(f'\n{i} - vector store "{vs.name}({vs.id})" has been deleted')

1 vector stores will now be deleted

1 - vector store "my_vectorstore(vs_X2aflqg4DzzvhHhEjpgID5Q9)" has been deleted


In [14]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agents will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")

1 agents will now be deleted

1 - Agent file-agent has been deleted


# HIC SUNT LEONES